In [1]:
import requests
import pandas as pd

In [2]:
api_key = 'Key'

# 소환사 데이터 수집(Challenger, GrandMaster, Master)

In [3]:
def get_league_data(league):
    url = 'https://kr.api.riotgames.com/lol/league/v4/'+ league +'leagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
    league_df = pd.DataFrame(requests.get(url).json())
    league_df.reset_index(inplace=True)#수집한 그마데이터 index정리
    league_entries_df = pd.DataFrame(dict(league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
    league_df = pd.concat([league_df, league_entries_df], axis=1) #열끼리 결합
    league_df = league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
#     league_df.to_csv('./data/' + league + '.csv',index=False,encoding = 'cp949')#중간저장
    return league_df

In [4]:
chal_df = get_league_data('challenger')
gma_df = get_league_data('grandmaster')
ma_df = get_league_data('master')

In [5]:
chal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tier          300 non-null    object
 1   summonerId    300 non-null    object
 2   summonerName  300 non-null    object
 3   leaguePoints  300 non-null    object
 4   wins          300 non-null    object
 5   losses        300 non-null    object
 6   veteran       300 non-null    object
 7   inactive      300 non-null    object
 8   freshBlood    300 non-null    object
 9   hotStreak     300 non-null    object
dtypes: object(10)
memory usage: 23.6+ KB


In [6]:
chal_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,Zg2epXxC2c0ObXJHfIX1eUmW2f2bJsV45oA8rnpmGALZuj4,익산누누,853,507,455,False,False,True,False
1,CHALLENGER,MEgpcYnO3UvT6gjZ5HxWXH_2c6JEv7oNBu8vsSA9sHhOIfk,진철수삼촌,905,325,275,False,False,True,True
2,CHALLENGER,uTU60AeWs0kKO9AzOAJdsSUnWH-OUXquYqaoIlMcH0Al67E,huntiand1,889,868,808,True,False,False,False
3,CHALLENGER,1icxPO3y_7wc4riQPIMnErH6bMJlSCNh-HR1oJkpGWKI7-A,Chieftain00,1173,298,227,True,False,False,False
4,CHALLENGER,17dXyYuKyxKncQ2U-taCYp2n9g16FbBpxGDmMt8ZB0pvVxE,11baby,835,1351,1269,True,False,False,False


# 유저별 고유 accountId 수집

In [7]:
def get_accountId(league_df):
    for i in range(len(league_df)):
        try:
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)

            while r.status_code == 429:
                time.sleep(5)
                sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
                r = requests.get(sohwan)

            account_id = r.json()['accountId']
            league_df.iloc[i, -1] = account_id

        except:
            pass

In [8]:
get_accountId(chal_df)

In [9]:
get_accountId(gma_df)

In [10]:
get_accountId(ma_df)

In [12]:
chal_df.head()

,tier,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,CHALLENGER,Zg2epXxC2c0ObXJHfIX1eUmW2f2bJsV45oA8rnpmGALZuj4,익산누누,853,507,455,False,False,True,MEhh2UIjggs8oEopLmFoYlicdqqqE8OZsA79439kyTjVyos
1,CHALLENGER,MEgpcYnO3UvT6gjZ5HxWXH_2c6JEv7oNBu8vsSA9sHhOIfk,진철수삼촌,905,325,275,False,False,True,6Xe4af-6MuxC_18ZDJhKhoYgc36tcx2cSX7u5movjNQcGiI
2,CHALLENGER,uTU60AeWs0kKO9AzOAJdsSUnWH-OUXquYqaoIlMcH0Al67E,huntiand1,889,868,808,True,False,False,U2SXHzoWtgP6YHrQE29GH5Ny-W-jXtKXqgemtCQzFlDndc...
3,CHALLENGER,1icxPO3y_7wc4riQPIMnErH6bMJlSCNh-HR1oJkpGWKI7-A,Chieftain00,1173,298,227,True,False,False,2WwJX6APhnKy88CuV7IuQI4uXqhyUunBYWW7j_rN1Mm1mKw
4,CHALLENGER,17dXyYuKyxKncQ2U-taCYp2n9g16FbBpxGDmMt8ZB0pvVxE,11baby,835,1351,1269,True,False,False,yCGf4c9EQ4sA6CJ2AYiCjqqs-NpKpzdryqPsAQknHVSCnL...
